In [7]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import accuracy_score
import csv
from sklearn.feature_extraction.text import CountVectorizer
import tweepy
from nltk.tokenize import TweetTokenizer
import json

auth = tweepy.OAuthHandler('4VvS7ZE84ka02TqtxeNgf1gvq', 'AOSg6cV7pm9gZqvWxCSrctcTwNoo5ZnhgjA04KIvbRuo20w5It')
auth.set_access_token('2668704372-erQWOs8zPPdrW8HVPibsFcLTCB4aOTFiE5n7xbA', 'BYhhFwimMQyDt6JNavggmY696SfjVQSdnDxWkj7qjzi3A')

# Preprocess text data

## Import data

In [19]:
api = tweepy.API(auth)

df2 = pd.read_csv('twitter_labelled/labeled_data.csv', encoding='utf-8')
ids = list(df2['ID'])
corpus = []
i = 0
Y = []

while i <= 3900:
    statuses = api.statuses_lookup(id_=ids[i: i+10])
    for ss in statuses:
        Y.append(df2['label'][ids.index(ss.id)])
    corpus += [s.text for s in statuses]
    i += 10

In [20]:
d = {'TweetText': corpus, 'label': Y}
df = pd.DataFrame(data=d)

In [21]:
df

,TweetText,label
0,Rutgers coach Kyle Flood says he has a 'good r...,0
1,Im glad coach Kirk Flood will lead us to victo...,0
2,Water main break floods parts of UCLA http://t...,1
3,"Teringat mee udang penang (@ Mee Udang Banjir,...",0
4,Rutgers training camp: Kyle Flood cautiously o...,0
5,Severe thunderstorm warning and flash flood wa...,0
6,"A picture on the wall in the Hale Center, home...",0
7,Paip air pecah menyebabkan banjir di Universit...,1
8,Rutgers coach Kyle Flood all smiles as his pla...,0
9,Rutgers training camp: 5 position battles to w...,0


## Remove URL, RT, mention(@)

In [55]:
df['ProcessedText'] = df['TweetText'].str.replace(r'http(\S)+', r'')
df['ProcessedText'] = df['ProcessedText'].str.replace(r'http ...', r'')

In [56]:
df['ProcessedText'][df['ProcessedText'].str.contains(r'http')]

Series([], Name: ProcessedText, dtype: object)

In [61]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'(RT|rt)[ ]*@[ ]*[\S]+',r'')

In [62]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'@',r'')
df['ProcessedText'][df['ProcessedText'].str.contains(r'@')]

Series([], Name: ProcessedText, dtype: object)

In [63]:
df['ProcessedText'][df['ProcessedText'].str.contains(r'RT[ ]?@')]

Series([], Name: ProcessedText, dtype: object)

In [64]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'@[\S]+',r'')

In [65]:
df

,TweetText,label,ProcessedText
0,Rutgers coach Kyle Flood says he has a 'good r...,0,Rutgers coach Kyle Flood says he has a 'good r...
1,Im glad coach Kirk Flood will lead us to victo...,0,Im glad coach Kirk Flood will lead us to victo...
2,Water main break floods parts of UCLA http://t...,1,Water main break floods parts of UCLA #Cairo ...
3,"Teringat mee udang penang (@ Mee Udang Banjir,...",0,"Teringat mee udang penang ( Mee Udang Banjir, ..."
4,Rutgers training camp: Kyle Flood cautiously o...,0,Rutgers training camp: Kyle Flood cautiously o...
5,Severe thunderstorm warning and flash flood wa...,0,Severe thunderstorm warning and flash flood wa...
6,"A picture on the wall in the Hale Center, home...",0,"A picture on the wall in the Hale Center, home..."
7,Paip air pecah menyebabkan banjir di Universit...,1,Paip air pecah menyebabkan banjir di Universit...
8,Rutgers coach Kyle Flood all smiles as his pla...,0,Rutgers coach Kyle Flood all smiles as his pla...
9,Rutgers training camp: 5 position battles to w...,0,Rutgers training camp: 5 position battles to w...


## Remove non-ascii words or characters

In [66]:
df['ProcessedText'] = [''.join([i if ord(i) < 128 else '' for i in text]) for text in df['ProcessedText']]

In [67]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'_[\S]?',r'')

## Remove extra space

In [68]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'[ ]{2, }',r' ')

## &, < and >

In [69]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'&amp;?',r'and')

In [70]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'&lt;',r'<')
df['ProcessedText'] = df['ProcessedText'].str.replace(r'&gt;',r'>')

## Insert space between words and punctuation marks

In [71]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'([\w\d]+)([^\w\d ]+)', r'\1 \2')

In [72]:
df['ProcessedText'] = df['ProcessedText'].str.replace(r'([^\w\d ]+)([\w\d]+)', r'\1 \2')

## Lowercased and strip

In [73]:
df['ProcessedText'] = df['ProcessedText'].str.lower()

In [74]:
df['ProcessedText'] = df['ProcessedText'].str.strip()

## Calculate text length for later use in LSTM

In [75]:
df['ProcessedText_len'] = [len(text.split(' ')) for text in df['ProcessedText']]

In [77]:
df['ProcessedText_len'].value_counts()

20    279
19    263
21    237
17    232
18    221
16    203
15    203
22    198
14    190
23    184
13    170
11    164
12    157
9     137
10    134
24    121
8      99
25     95
7      91
6      74
26     71
5      65
27     57
4      37
28     28
3      25
29     14
30     12
31      7
2       6
32      4
1       2
37      2
35      2
50      1
33      1
43      1
Name: ProcessedText_len, dtype: int64

## Drop texts with length <=3 and drop duplicates

In [80]:
df = df[df['ProcessedText_len']>3]

In [81]:
df = df.drop_duplicates(subset=['ProcessedText'])

## Summary of sample size and labels

In [82]:
df.shape[0]

3749

## BERT preprocess

In [83]:
df['ProcessedText_BERT'] = '[CLS] '+ df['ProcessedText']

In [86]:
from pytorch_pretrained_bert import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
df['ProcessedText_BERTbase_length'] = [len(tokenizer.tokenize(sent)) for sent in df['ProcessedText_BERT']]

tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
df['ProcessedText_BERTlarge_length'] = [len(tokenizer.tokenize(sent)) for sent in df['ProcessedText_BERT']]

100%|██████████| 231508/231508 [00:00<00:00, 1422224.04B/s]


## Int label for later use in softmax and cross entropy loss

In [89]:
# label_dict = dict()
# for i, l in enumerate(list(df.InformationType_coarse.value_counts().keys())):
#     label_dict.update({l: i})

# df['label'] = [label_dict[label] for label in df.InformationType_coarse]


## Save data

In [90]:
df.to_csv('processed_tweets.csv')